<a href="https://colab.research.google.com/github/PikoAll/machine-learning-e-deep-learning/blob/main/AIFishTensori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
# per usare direttamente il dataset di kaggle
import os

user='pikoall'
key='ff6917929b2de4be4c90d80407661b0f'
if '.kaggle' not in os.listdir('/root'):
  !mkdir ~/.kaggle
!touch /root/.kaggle/kaggle.json
!chmod 666 /root/.kaggle/kaggle.json
with open('/root/.kaggle/kaggle.json','w') as f:
  f.write('{"username":"%s","key":"%s"}'%(user,key))
!chmod 600 /root/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d crowww/a-large-scale-fish-dataset

 99% 3.22G/3.24G [00:24<00:00, 139MB/s]
100% 3.24G/3.24G [00:24<00:00, 139MB/s]


In [ ]:
os.mkdir('Dataset')

In [ ]:
import zipfile
archivio = zipfile.ZipFile('/content/a-large-scale-fish-dataset.zip')

# estrazione di tutti i file
archivio.extractall("/content/Dataset/")  #percorso dove estrerre i file

# chiusura
archivio.close()

In [ ]:
#elimino alcuni file inutili
try: 
  os.remove("/content/Dataset/Fish_Dataset/Fish_Dataset/README.txt")
  #shutil.rmtree("/content/Dataset/Fish_Dataset/Fish_Dataset/Segmentation_example_script.m")
  os.remove("/content/Dataset/Fish_Dataset/Fish_Dataset/license.txt")
except EOFError:
  print("problemi a eliminare qualcosa")


os.chdir("/content/Dataset/Fish_Dataset/Fish_Dataset")
print(len(os.listdir()))
os.listdir()

10


['Shrimp',
 'Red Mullet',
 'Trout',
 'Hourse Mackerel',
 'Striped Red Mullet',
 'Segmentation_example_script.m',
 'Black Sea Sprat',
 'Red Sea Bream',
 'Gilt-Head Bream',
 'Sea Bass']

#carico il dataset utilizzando keras

In [ ]:
import pathlib
import tensorflow as tf

È buona norma utilizzare una suddivisione di convalida durante lo sviluppo del modello. Usiamo l'80% delle immagini per l'addestramento e il 20% per la convalida.

In [ ]:
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  '/content/Dataset/Fish_Dataset/Fish_Dataset/',
  validation_split=0.3,
  subset="training",
  seed=123,
  color_mode='rgb',
  image_size=(img_height, img_width),
  batch_size=batch_size
  )

Found 18000 files belonging to 9 classes.
Using 12600 files for training.


In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  '/content/Dataset/Fish_Dataset/Fish_Dataset/',
  validation_split=0.3,
  subset="validation",
  seed=123,
  color_mode='rgb',
  image_size=(img_height, img_width),
  batch_size=batch_size
  )

Found 18000 files belonging to 9 classes.
Using 5400 files for validation.


In [ ]:
class_names = train_ds.class_names
print(class_names)
print(len(class_names))

['Black Sea Sprat', 'Gilt-Head Bream', 'Hourse Mackerel', 'Red Mullet', 'Red Sea Bream', 'Sea Bass', 'Shrimp', 'Striped Red Mullet', 'Trout']
9


In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

tf.Tensor(
[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  ...

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]


 [[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  ...

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
   ...
   [0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]
 

Dataset.cache mantiene le immagini nella memoria dopo che sono stati caricati dal disco durante la prima epoca. Ciò garantirà che il set di dati non diventi un collo di bottiglia durante l'addestramento del modello. Se il tuo set di dati è troppo grande per entrare in memoria, puoi anche usare questo metodo per creare una cache su disco performante.
Dataset.prefetch sovrappone i dati di pre-elaborazione e l'esecuzione del modello, mentre la formazione.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
num_classes = 9
'''
model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])
'''

from tensorflow.keras.utils import  to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Dropout,Conv2D,MaxPool2D, BatchNormalization, MaxPooling2D, Rescaling   ############################# o 3D
from tensorflow.keras.regularizers import l2 
import tensorflow
from sklearn.metrics import confusion_matrix

from tensorflow.keras import backend as K
'''
model=tf.keras.Sequential([
tf.keras.layers.Rescaling(1./255),
Conv2D(32, kernel_size=(3, 3), activation='relu') ,  #########################
MaxPooling2D(pool_size=(2, 2)),
Conv2D(64, kernel_size=(3, 3), activation='relu'),
MaxPooling2D(pool_size=(2, 2)),
Flatten(),
Dense(254, activation='relu',kernel_regularizer=l2(0.01)),
Dropout(0.5),
Dense(64, activation='relu',kernel_regularizer=l2(0.01)),
Dropout(0.5),
Dense(10)

])
'''

from tensorflow.keras import backend as K
K.clear_session()
model = Sequential()
model.add(Rescaling(1./255))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', data_format='channels_last'))   #########################
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(254, activation='relu',kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu',kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(9, activation='softmax'))


In [ ]:
#model.compile(loss=tensorflow.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer=tensorflow.keras.optimizers.SGD(0.01), metrics=["accuracy","AUC"]) #SparseCategoricalCrossentropy()   ooookkkkkkk

model.compile(
  optimizer=tensorflow.keras.optimizers.SGD(0.01),
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=30,
)

test_results = model.evaluate(val_ds)
print('\nTest Acc. {:.2f}%'.format(test_results[1]*100))

test_results = model.evaluate(train_ds)
print('\nTrain Acc. {:.2f}%'.format(test_results[1]*100))
'''
pred=model.predict(X_test)
pred=np.argmax(pred,axis=1)


metrics_train=model.evaluate(X_train,y_train)
metrics_test=model.evaluate(X_test,y_test)
print("Train accuracy ",metrics_train[1])
print("Test accuracy ",metrics_test[1])
'''

Epoch 1/30


/usr/local/lib/python3.7/dist-packages/keras/backend.py:4907: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


394/394 [==============================] - 13s 32ms/step - loss: 7.7901 - accuracy: 0.1640 - val_loss: 7.1963 - val_accuracy: 0.3104
Epoch 2/30
 99/394 [======>.......................] - ETA: 8s - loss: 7.1177 - accuracy: 0.2509

KeyboardInterrupt: ignored

In [ ]:
'''
def preprocess(item):
  image=item['image']
  label=item['label']
  image=tf.image.convert_image_dtype(image,tf.float32)
  image=tf.reshape(image,(-1,))
  return {'image-pixels':image}, label[..., tf.newaxis]

#-------------------------------------------------
BUFFER_SIZE = 10000
BATCH_SIZE = 64
NUM_EPOCHS = 20

mnist_train = mnist_train_orig.map(
    lambda item: (tf.cast(item['image'], tf.float32)/255.0, 
                  tf.cast(item['label'], tf.int32)))

mnist_test = mnist_test_orig.map(
    lambda item: (tf.cast(item['image'], tf.float32)/255.0, 
                  tf.cast(item['label'], tf.int32)))

tf.random.set_seed(1)

mnist_train = mnist_train.shuffle(buffer_size=BUFFER_SIZE,
                                  reshuffle_each_iteration=False)

mnist_valid = mnist_train.take(10000).batch(BATCH_SIZE)
mnist_train = mnist_train.skip(10000).batch(BATCH_SIZE)
'''